# Agentic RAG with LlamaIndex


In this notebook we will experiment the agent worker and runner concept.

- Define a reader to read the `pdf` sample file [AraGPT2](./data/aragpt2.pdf) paper.
- Define a `splitter` to process the texts of the document.
- Set the LLM embedding and generation model ids.
- Create the engines from the Indexes and define a tool wrapper around them.
- Define the agent worker and agent runner that utilize memory.
- Debug the results by manually excuting the tasks.


## Setups


In [1]:
from rich import print
from dotenv import load_dotenv

In [2]:
# load env variables
_ = load_dotenv()

In [3]:
# define some constants
GENERATION_MODEL_ID = "gpt-4o-mini"
EMBEDDING_MODEL_ID = "text-embedding-3-small"

## Load Documents


In [4]:
from llama_index.core import SimpleDirectoryReader

documents_reader = SimpleDirectoryReader(input_files=["./data/aragpt2.pdf"])
documents = documents_reader.load_data()

In [5]:
print(documents[0])

Document(
    id_='a119103a-0c4a-4a60-affb-6bce7c5e7bf4',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    text='Proceedings of the Sixth Arabic Natural Language Processing Workshop , pages 196–207\nKyiv, Ukraine 
(Virtual), April 19, 2021.196ARAGPT2: Pre-T rained Transformer\nfor Arabic Language Generation\nWissam Antoun and 
Fady Baly and Hazem Hajj\nAmerican University of Beirut\n{wfa07, fbg06, hh63 }@aub.edu.lb\nAbstract\nRecently, 
pre-trained transformer-based archi-\ntectures have proven to be very efﬁcient atlanguage modeling and 
understanding, giventhat they are trained on a large enough cor-pus. Applications in language generation 
for\nArabic are still lagging in comparison to other\nNLP advances primarily due to the lack ofadvanced Arabic 
language generation models.In this paper, we develop the ﬁrst advanced\nArabic language generation model, 
AraGPT2,\ntrained from scratch on a large Arabic corpusof internet text and news articles. Our largest\nmodel, 
A\nRAGPT2- MEGA , has 1.46 billion pa-\nrameters, which makes it the largest Arabiclanguage model available. 
The\nMEGA model\nwas evaluated and showed success on different\ntasks including synthetic news generation, 
andzero-shot question answering. For text gener-\nation, our best model achieves a perplexity of\n29.8 on held-out 
Wikipedia articles. A studyconducted with human evaluators showed thesigniﬁcant success of AraGPT2-mega in 
gen-erating news articles that are difﬁcult to dis-tinguish from articles written by humans. Wethus develop and 
release an automatic discrim-inator model with a 98% percent accuracy indetecting model-generated text. The 
modelsare also publicly available\n1, hoping to encour-\nage new research directions and applicationsfor Arabic 
NLP.\n1 Introduction\nFew years ago, Natural language processing\n(NLP) was revolutionized with the 
introduction\nof multi-head self-attention transformer architec-ture ( Vaswani et al. ,2017 ). The 
transformer\nachieved superior performance compared to recur-\nrent neural networks several NLP tasks 
including\nmachine translation, sentence classiﬁcation with\n1Pretrained variants of A RAGPT2 (base, 
medium,\nlarge, mega) and discriminator are publicly available 
on\ngithub.com/aub-mind/arabert/tree/master/aragpt2BERT ( Devlin et al. ,2019 ), and ELECTRA ( Clark\net al. ,2020b
), and sentence completion with GPT-\n2(Radford et al. ,2019 ), GROVER ( Zellers et al. ,\n2019 ), and CTRL ( 
Keskar et al. ,2019 ). Recent\nworks have shown that larger models pre-trained\non larger datasets can further 
improve performancei.e. RoBERTa ( Liu et al. ,2019 ), and XLM-R ( Con-\nneau et al. ,2019 ).\nOn the other hand, 
work on Arabic language\nmodeling has mostly targeted natural language\nunderstanding (NLU) by pre-training 
transformer-\nbased models using the Masked Language Model-\ning (MLM) task i.e. A RABERT ( Antoun et al. ,\n2020a 
). In contrast, Arabic text generation or\ncausal language modeling hasn’t received much\nattention. Few works such
as hULMonA ( ElJundi\net al. ,2019 ) used next word prediction as a pre-\ntraining task in for transfer learning in
Arabic text\nclassiﬁcation. ( Khooli ,2020 ) and ( Doiron ,2020 )\nleveraged the existing GPT2 English model 
andadapted it for Arabic using text from the Arabic\nWikipedia dumps, which is sub-optimal for Arabic

In [6]:
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=32)
nodes = sentence_splitter.get_nodes_from_documents(documents)

In [7]:
print(len(nodes))
print(nodes[2])

89

TextNode(
    id_='95a50672-6982-4e4a-abcd-ae36a1659daa',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={
        <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
            node_id='a119103a-0c4a-4a60-affb-6bce7c5e7bf4',
            node_type=<ObjectType.DOCUMENT: '4'>,
            metadata={
                'page_label': '1',
                'file_name': 'aragpt2.pdf',
                'file_path': 'data/aragpt2.pdf',
                'file_type': 'application/pdf',
                'file_size': 457776,
                'creation_date': '2024-11-12',
                'last_modified_date': '2024-10-22'
            },
            hash='be467e8635c423cf5e905a9469550ed6e5194fba229387b29d1ef0cf335d08de'
        ),
        <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
            node_id='80e1413e-9f46-4cbb-8de5-9d4885461c9f',
            node_type=<ObjectType.TEXT: '1'>,
            metadata={
                'page_label': '1',
                'file_name': 'aragpt2.pdf',
                'file_path': 'data/aragpt2.pdf',
                'file_type': 'application/pdf',
                'file_size': 457776,
                'creation_date': '2024-11-12',
                'last_modified_date': '2024-10-22'
            },
            hash='d29e9f18057bb077fc974a135476bbfa465cfbf92c45fa06545fe29f0a9eb880'
        )
    },
    text='The process of pre-\ntraining A RAGPT2, a GPT -2 transformer model\nfor the Arabic language is described.
The modelcomes in 4 size variants:\nbase (135M2),medium\n(370M), large (792M) and mega (1.46B3), which\nallows the 
exploration of A RAGPT2 in multiple ap-\nplications with different data availability and com-\nputational 
constraints. The perplexity measure is\nused to automatically evaluate A RAGPT2. Fur-\nthermore, a human-based 
evaluation is provided,\nwhich highlights the ability of A RAGPT2 to de-\nceive human evaluators. Finally, an A 
RAELEC-\nTRA ( Antoun et al. ,2020b ) based detector is devel-\n2Million Parameters\n3Billion Parameters',
    mimetype='text/plain',
    start_char_idx=3290,
    end_char_idx=3930,
    text_template='{metadata_str}\n\n{content}',
    metadata_template='{key}: {value}',
    metadata_seperator='\n'
)

## Define Backend Settings


In [8]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model=GENERATION_MODEL_ID)
Settings.embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL_ID)

## Define Vector and Summary Indecies


In [9]:
from llama_index.core import VectorStoreIndex, SummaryIndex

vector_index = VectorStoreIndex(nodes=nodes)
summary_index = SummaryIndex(nodes=nodes)

## Convert Indecies to Tools


In [10]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [11]:
vector_engine = vector_index.as_query_engine()
vector_tool = QueryEngineTool(
    query_engine=vector_engine,
    metadata=ToolMetadata(
        name="vector_tool",
        description="Useful for retrieving specific context from the aragpt2 paper.",
    ),
)

In [12]:
summary_engine = summary_index.as_query_engine(response_mode="tree_summarize")
summary_tool = QueryEngineTool(
    query_engine=summary_engine,
    metadata=ToolMetadata(
        name="summary_tool",
        description="Useful for summarization questions related to the aragpt2 paper.",
    ),
)

## Define Agent Worker and Runner


In [13]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[vector_tool, summary_tool], verbose=True
)
agent_runner = AgentRunner(agent_worker=agent_worker)

In [14]:
query = "What datasets were used to train the AraGPT2 model?"
response = agent_runner.chat(query)

Added user message to memory: What datasets were used to train the AraGPT2 model?
=== Calling Function ===
Calling function: vector_tool with args: {"input": "datasets used to train the AraGPT2 model"}
=== Function Output ===
The training dataset for the AraGPT2 model includes the following publicly available Arabic corpora:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles provided by As-safir newspaper

Additionally, the dataset underwent preprocessing, which involved filtering out short documents, removing repeated sentences, and replacing URLs, emails, and user mentions with special tokens, among other modifications.
=== LLM Response ===
The AraGPT2 model was trained on several publicly available Arabic corpora, including:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles from the As-safir newsp

In [15]:
query = "What was the combined size in GBs of those data?"
response = agent_runner.chat(query)

Added user message to memory: What was the combined size in GBs of those data?
=== Calling Function ===
Calling function: vector_tool with args: {"input": "combined size in GBs of datasets used to train AraGPT2 model"}
=== Function Output ===
The combined size in GBs of the datasets used to train the AraGPT2 model is not specified in the provided information.
=== LLM Response ===
The combined size in GBs of the datasets used to train the AraGPT2 model is not specified in the available information.


In [16]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 4
file_name: aragpt2.pdf
file_path: data/aragpt2.pdf
file_type: application/pdf
file_size: 457776
creation_date: 2024-11-12
last_modified_date: 2024-10-22

199Model Size Architecture Context Size Emb. Size Heads Layers Optimizer
Base 135M GPT2 1024 768 12 12 LAMB
Medium 370M GPT2 1024 1024 16 24 LAMB
Large 792M GROVER 1024 1280 20 36 Adafactor
Mega 1.46B GROVER 1024 1536 24 48 Adafactor
Table 1: A RAGPT2 model variants with sizes, architecture and optimizer
Model Batch Size Learning Rate Steps Time (days) PPL
Base 1792 1.27e-3 120K 1.5 55.8
Medium* 80 3e-4 1M 23 45.7
Large 256 1e-4 220K 3 36.6
Mega 256 1e-4 780K 9 29.8
Table 2: A RAGPT2 training details and validation perplexity. * Medium was trained on a TPUv3-8 with a small
batch size, since the model was not converging with a large batch size
4 Experiments and Evaluation
4.1 Pre-training Setup
All models were trained on a TPUv3-128 slice9
with different batch sizes and the total number
of steps as shown in Table 2.Base and mega
were trained for approximately 20 epochs, while
medium and large were trained for 10 and 6
epochs respectively, due to TPU access limitations.
4.2 Numerical Evaluation
For the validation dataset, the Arabic Wikipedia
articles that were published after August 2020
were used, since older articles were included in
the September Wikipedia dump. The perplexity
score was selected as a numerical evaluation met-
ric since it measures the degree of ’uncertainty’ a
model has assigning probabilities to the test text.
Table 2shows that, unsurprisingly, validation per-
plexity keeps improving with larger model sizes.
In fact, the model is still under-ﬁtting the validation
set from Wikipedia. The generation capabilities of
the different variants of A RAGPT2 is illustrated
through the selected examples in Appendix A.

## Agent Debugging


In [17]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[vector_tool, summary_tool], verbose=True
)
agent_runner = AgentRunner(agent_worker=agent_worker)

In [18]:
query = "What datasets were used to train the AraGPT2 models and what evaluation methods were used?"

In [19]:
task = agent_runner.create_task(input=query)

In [20]:
print(task)

Task(
    task_id='d916bd52-8965-493e-8280-2e412525ed9a',
    input='What datasets were used to train the AraGPT2 models and what evaluation methods were used?',
    memory=ChatMemoryBuffer(
        chat_store=SimpleChatStore(store={}),
        chat_store_key='chat_history',
        token_limit=3000,
        tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, 
allowed_special='all')
    ),
    callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fb8a4b794d0>,
    extra_state={
        'sources': [],
        'n_function_calls': 0,
        'new_memory': ChatMemoryBuffer(
            chat_store=SimpleChatStore(store={}),
            chat_store_key='chat_history',
            token_limit=3000,
            tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, 
allowed_special='all')
        )
    }
)

In [21]:
step_output = agent_runner.run_step(task_id=task.task_id)

Added user message to memory: What datasets were used to train the AraGPT2 models and what evaluation methods were used?
=== Calling Function ===
Calling function: vector_tool with args: {"input": "datasets used to train AraGPT2 models"}
=== Function Output ===
The training dataset for the AraGPT2 models includes the following publicly available Arabic corpora:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles provided by As-safir newspaper

Additionally, the dataset was preprocessed by filtering out short documents, removing repeated sentences, and replacing URLs, emails, and user mentions with special tokens, among other modifications.
=== Calling Function ===
Calling function: vector_tool with args: {"input": "evaluation methods used for AraGPT2 models"}
=== Function Output ===
The evaluation methods used for AraGPT2 models include an automatic evaluation based on the perplexity measure, a

In [22]:
completed_steps = agent_runner.get_completed_steps(task_id=task.task_id)

In [23]:
print(len(completed_steps))

1

In [24]:
upcoming_steps = agent_runner.get_upcoming_steps(task_id=task.task_id)

In [25]:
print(upcoming_steps)

[
    TaskStep(
        task_id='d916bd52-8965-493e-8280-2e412525ed9a',
        step_id='c2e06854-221b-46d9-8d3e-33a254b1443f',
        input=None,
        step_state={},
        next_steps={},
        prev_steps={},
        is_ready=True
    )
]

In [26]:
step_output = agent_runner.run_step(task_id=task.task_id)

=== LLM Response ===
The AraGPT2 models were trained using several publicly available Arabic corpora, including:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles provided by As-safir newspaper

The dataset was preprocessed by filtering out short documents, removing repeated sentences, and replacing URLs, emails, and user mentions with special tokens, among other modifications.

For evaluation, the AraGPT2 models employed two methods:

1. An automatic evaluation based on the perplexity measure.
2. A human-based evaluation that assesses the model's ability to deceive human evaluators.


In [27]:
print(step_output)

TaskStepOutput(
    output=AgentChatResponse(
        response="The AraGPT2 models were trained using several publicly available Arabic corpora, including:\n\n- 
The unshuffled OSCAR corpus\n- The Arabic Wikipedia dump from September 2020\n- The 1.5B words Arabic Corpus\n- The
OSIAN corpus\n- News articles provided by As-safir newspaper\n\nThe dataset was preprocessed by filtering out short
documents, removing repeated sentences, and replacing URLs, emails, and user mentions with special tokens, among 
other modifications.\n\nFor evaluation, the AraGPT2 models employed two methods:\n\n1. An automatic evaluation 
based on the perplexity measure.\n2. A human-based evaluation that assesses the model's ability to deceive human 
evaluators.",
        sources=[],
        source_nodes=[],
        is_dummy_stream=False,
        metadata=None
    ),
    task_step=TaskStep(
        task_id='d916bd52-8965-493e-8280-2e412525ed9a',
        step_id='c2e06854-221b-46d9-8d3e-33a254b1443f',
        input=None,
        step_state={},
        next_steps={},
        prev_steps={},
        is_ready=True
    ),
    next_steps=[],
    is_last=True
)

In [28]:
agent_runner.get_upcoming_steps(task_id=task.task_id) # no upcomming steps

[]

In [29]:
response = agent_runner.finalize_response(task_id=task.task_id)
print(response.response)

The AraGPT2 models were trained using several publicly available Arabic corpora, including:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles provided by As-safir newspaper

The dataset was preprocessed by filtering out short documents, removing repeated sentences, and replacing URLs, 
emails, and user mentions with special tokens, among other modifications.

For evaluation, the AraGPT2 models employed two methods:

1. An automatic evaluation based on the perplexity measure.
2. A human-based evaluation that assesses the model's ability to deceive human evaluators.